In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import pandas as pd
import random
import os
from tqdm import tqdm
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data import ConcatDataset
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, OneCycleLR, CosineAnnealingWarmRestarts, StepLR
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
import cv2

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_base_dir = '/data/ephemeral/home/data/train'
test_base_dir = '/data/ephemeral/home/data/test'


traindata_info_file = "/data/ephemeral/home/data/train.csv"

testdata_info_file = "/data/ephemeral/home/data/test.csv"

train_data = pd.read_csv(traindata_info_file)

test_data = pd.read_csv(testdata_info_file)

x = train_data['image_path']
y = train_data['target']

train_data.head(3)

,class_name,image_path,target
0,n01872401,n01872401/sketch_50.JPEG,59
1,n02417914,n02417914/sketch_11.JPEG,202
2,n02106166,n02106166/sketch_3.JPEG,138


In [ ]:
class Train():
    def __init__(self, model, device, train_loader, val_loader, epochs, optimizer, criterion, scheduler,
                 early_stop = False, patience_limit = None, best_val_loss = float('inf'), best_model = None):
        self.model = model
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.epochs = epochs
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = scheduler
        self.early_stop = early_stop
        self.patience_limit = patience_limit
        self.best_val_loss = best_val_loss
        self.best_model = best_model

    def train(self):
        patience_check = 0
        self.model.to(self.device)
        for epoch in range(self.epochs):
            running_loss = 0.0

            # 모델 학습
            torch.cuda.empty_cache()
            self.model.train()
            for idx, (images, labels) in enumerate(tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.epochs}")):
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(self.train_loader)

            # 모델 평가
            torch.cuda.empty_cache()
            self.model.eval()
            correct = 0
            total = 0
            running_val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in self.val_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)

                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)
                    running_val_loss += loss.item()
                    _, pred = torch.max(outputs, 1)

                    total += labels.size(0)
                    correct += (pred == labels).sum().item()

                accuracy = 100 * correct / total
                val_loss = running_val_loss / len(self.val_loader)

            print(f'Epoch {epoch + 1}/{self.epochs}, Train_Loss: {train_loss:.4f}, Test_Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

            if self.best_val_loss > val_loss:
                self.best_val_loss = val_loss
                self.best_model = self.model.state_dict()
                torch.save(self.model.state_dict(), '/data/ephemeral/home/jb_train_result/Clip_lr.pt')

            # 조기 종료
            if self.early_stop and self.patience_limit is not None:
                if val_loss > self.best_val_loss:
                    patience_check += 1
                    if patience_check >= self.patience_limit:
                        break
                else:
                    self.best_val_loss = val_loss
                    patience_check = 0
                    torch.save(self.model.state_dict(), '/data/ephemeral/home/jb_train_result/Clip_lr.pt')

            self.scheduler.step(val_loss)

In [ ]:
transform = A.Compose([
        A.Resize(224,224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

In [ ]:
def fill_white(img):
    img_np = np.array(img)

    # 각 채널에 대해 Canny 엣지 검출 적용
    edges_r = cv2.Canny(img_np[:, :, 0], 50, 150)
    edges_g = cv2.Canny(img_np[:, :, 1], 50, 150)
    edges_b = cv2.Canny(img_np[:, :, 2], 50, 150)

    # 세 채널의 엣지를 결합하여 하나의 이미지로 생성
    edges_combined = np.maximum(np.maximum(edges_r, edges_g), edges_b)

    # 모폴로지 연산을 위한 커널 생성
    kernel = np.ones((3, 3), np.uint8)
    # 엣지 이미지를 닫기 위한 모폴로지 연산 적용
    closed_edges = cv2.morphologyEx(edges_combined, cv2.MORPH_CLOSE, kernel)

    filled_image_rgb = cv2.cvtColor(closed_edges, cv2.COLOR_GRAY2RGB)

    inverted_edges = cv2.bitwise_not(closed_edges)

    # 흰색 배경 이미지 생성
    filled_image_rgb = np.full_like(img_np, 255)
    # 배경이 흰색인 상태에서 검은색 선을 포함한 inverted_edges를 복사
    filled_image_rgb[inverted_edges == 0] = [0, 0, 0]

    transform = A.Compose([
        A.Affine(scale=(0.5, 1.5), p=0.5),
        A.CoarseDropout(max_holes=4, max_height=30, max_width=30, fill_value=255, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.LongestMaxSize(256),
        A.PadIfNeeded(256, 256, border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255)),
        A.RandomCrop(224, 224,p=1.0),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

    transformed = transform(image=filled_image_rgb)

    return transformed['image']


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, is_aug = False):
        self.image_paths = image_paths
        self.labels = labels
        self.is_aug = is_aug

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(os.path.join(train_base_dir, image_path)).convert('RGB')
        if not self.is_aug:
            image = np.array(image)
            image = transform(image=image)['image']
        else:
            image = fill_white(image)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

In [ ]:
x_train = []
y_train = []
for i in range(len(train_data)):
    x_train.append(os.path.join(train_base_dir, train_data['image_path'].iloc[i]))
    y_train.append(train_data['target'].iloc[i])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify= y_train, random_state = 42)

print(len(train_data))
print(len(x_train))
print(len(x_val))

15021
12016
3005


In [ ]:
train_dataset = CustomDataset(x_train, y_train, is_aug = False)
aug_dataset = CustomDataset(x_train, y_train, is_aug = True)
val_dataset = CustomDataset(x_val, y_val, is_aug = False)

print(len(train_dataset))
print(len(aug_dataset))
print(len(val_dataset))

dataset = ConcatDataset([train_dataset, aug_dataset])
print(len(dataset))

12016
12016
3005
24032


In [ ]:
train_loader = DataLoader(dataset, 64, num_workers=4, shuffle=True)
val_loader = DataLoader(val_dataset, 64, num_workers=2, shuffle=False)

In [ ]:
from transformers import CLIPModel

clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
image_encoder = clip.vision_model

class Clip(nn.Module):
    def __init__(self, image_encoder):
        super(Clip, self).__init__()
        self.clip = image_encoder
        self.mlp = nn.Sequential(
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 500)
        )

    def forward(self, images):
        image_features = self.clip(images).last_hidden_state

        y = self.mlp(image_features[:, 0, :])
        return y


In [ ]:
def freeze(model):
    cnt=0
    for param in model.parameters():
        if cnt > 390:
            param.requires_grad=True
        else:
            param.requires_grad = False
        cnt+=1

In [ ]:
model = Clip(image_encoder)
freeze(model)

In [ ]:
for name, param in model.named_parameters():
    print(f'{name}: {"requires_grad=True" if param.requires_grad else "requires_grad=False"}')

clip.embeddings.class_embedding: requires_grad=False
clip.embeddings.patch_embedding.weight: requires_grad=False
clip.embeddings.position_embedding.weight: requires_grad=False
clip.pre_layrnorm.weight: requires_grad=False
clip.pre_layrnorm.bias: requires_grad=False
clip.encoder.layers.0.self_attn.k_proj.weight: requires_grad=False
clip.encoder.layers.0.self_attn.k_proj.bias: requires_grad=False
clip.encoder.layers.0.self_attn.v_proj.weight: requires_grad=False
clip.encoder.layers.0.self_attn.v_proj.bias: requires_grad=False
clip.encoder.layers.0.self_attn.q_proj.weight: requires_grad=False
clip.encoder.layers.0.self_attn.q_proj.bias: requires_grad=False
clip.encoder.layers.0.self_attn.out_proj.weight: requires_grad=False
clip.encoder.layers.0.self_attn.out_proj.bias: requires_grad=False
clip.encoder.layers.0.layer_norm1.weight: requires_grad=False
clip.encoder.layers.0.layer_norm1.bias: requires_grad=False
clip.encoder.layers.0.mlp.fc1.weight: requires_grad=False
clip.encoder.layers.0.

In [ ]:


epochs = 100
criterion =nn.CrossEntropyLoss(label_smoothing = 0.1)
optimizer = optim.AdamW(filter(lambda p:p.requires_grad,model.parameters()), lr=1e-3, weight_decay=0.01)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
trainer = Train(model, device = device, train_loader = train_loader, val_loader = val_loader, epochs = epochs,
                    optimizer = optimizer, criterion = criterion, scheduler = scheduler, early_stop= True, patience_limit=5)
trainer.train()

torch.save(trainer.model.state_dict(), '/data/ephemeral/home/jb_train_result/Clip_rop.pt')

Epoch 1/100: 100%|██████████| 376/376 [05:44<00:00,  1.09it/s]


Epoch 1/100, Train_Loss: 3.9397, Test_Loss: 2.0467, Accuracy: 72.88%


Epoch 2/100: 100%|██████████| 376/376 [05:44<00:00,  1.09it/s]


Epoch 2/100, Train_Loss: 2.5175, Test_Loss: 1.7830, Accuracy: 78.80%


Epoch 3/100: 100%|██████████| 376/376 [05:44<00:00,  1.09it/s]


...

Epoch 46/100, Train_Loss: 1.6982, Test_Loss: 1.5319, Accuracy: 86.26%


Epoch 47/100: 100%|██████████| 376/376 [05:45<00:00,  1.09it/s]


Epoch 47/100, Train_Loss: 1.6998, Test_Loss: 1.5314, Accuracy: 86.16%
Epoch 00047: reducing learning rate of group 0 to 1.0000e-06.


Epoch 48/100: 100%|██████████| 376/376 [05:45<00:00,  1.09it/s]


Epoch 48/100, Train_Loss: 1.6999, Test_Loss: 1.5310, Accuracy: 86.12%


In [ ]:
model.load_state_dict(torch.load('/data/ephemeral/home/jb_train_result/Clip_check.pt'))
torch.save(model.state_dict(), '/data/ephemeral/home/jb_train_result/Clip_check.pt')

In [ ]:
total_params = 397
params_per_group = 20
def set_param_groups(model, group_index, params_per_group):
    cnt = 0
    for param in model.parameters():
        # 해당 그룹의 파라미터만 고정 해제
        if group_index * params_per_group <= cnt < (group_index + 1) * params_per_group:
            param.requires_grad = True  # 해제
        else:
            param.requires_grad = False  # 고정
        cnt += 1
# 총 그룹 개수 계산 (나머지 파라미터 포함)
num_groups = (total_params + params_per_group - 1) // params_per_group  # 그룹 개수 계산 (올림)


In [ ]:
for group_index in range(num_groups):
    model.load_state_dict(torch.load('/data/ephemeral/home/jb_train_result/Clip_lr.pt'))
    set_param_groups(model,group_index,params_per_group)
    criterion =nn.CrossEntropyLoss(label_smoothing = 0.1)
    optimizer = optim.AdamW(filter(lambda p:p.requires_grad,model.parameters()), lr=1e-6, weight_decay=0.01)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    trainer = Train(model, device = device, train_loader = train_loader, val_loader = val_loader, epochs = 5,
                        optimizer = optimizer, criterion = criterion, scheduler = scheduler, early_stop= False, patience_limit=5)
    trainer.train()
    model.load_state_dict(torch.load('/data/ephemeral/home/jb_train_result/Clip_lr.pt'))
    torch.save(trainer.model.state_dict(), f'/data/ephemeral/home/jb_train_result/Clip_fine{group_index}.pt')

Epoch 1/5: 100%|██████████| 376/376 [07:52<00:00,  1.26s/it]


Epoch 1/5, Train_Loss: 1.5110, Test_Loss: 1.4856, Accuracy: 87.32%


Epoch 2/5: 100%|██████████| 376/376 [07:52<00:00,  1.26s/it]


Epoch 2/5, Train_Loss: 1.5184, Test_Loss: 1.4831, Accuracy: 87.29%


Epoch 3/5: 100%|██████████| 376/376 [07:52<00:00,  1.26s/it]


...

Epoch 3/5, Train_Loss: 1.4792, Test_Loss: 1.4722, Accuracy: 87.32%


Epoch 4/5: 100%|██████████| 376/376 [06:10<00:00,  1.02it/s]


Epoch 4/5, Train_Loss: 1.4810, Test_Loss: 1.4726, Accuracy: 87.39%


Epoch 5/5: 100%|██████████| 376/376 [06:10<00:00,  1.02it/s]


Epoch 5/5, Train_Loss: 1.4689, Test_Loss: 1.4732, Accuracy: 87.29%


In [ ]:
model.load_state_dict(torch.load('/data/ephemeral/home/jb_train_result/Clip_fine19.pt'))

<All keys matched successfully>

In [ ]:
class TestDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(os.path.join(test_base_dir, image_path)).convert('RGB')
        image = np.array(image)
        image = transform(image=image)['image']
        return image
test_dataset = TestDataset(list(test_data['image_path']))

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
print(len(test_loader))

157


In [ ]:
model.to(device)
model.eval()

predictions = []
with torch.no_grad():
    for images in tqdm(test_loader):
        images = images.to(device)

        logits = model(images)
        logits = F.softmax(logits, dim=1)
        preds = logits.argmax(dim=1)

        predictions.extend(preds.cpu().detach().numpy())

  0%|          | 0/157 [00:00<?, ?it/s]

100%|██████████| 157/157 [02:57<00:00,  1.13s/it]


In [ ]:
test_data1 = pd.read_csv(testdata_info_file)

test_data1['target'] = predictions
test_data1 = test_data1.reset_index().rename(columns={"index": "ID"})
test_data1

,ID,image_path,target
0,0,0.JPEG,328
1,1,1.JPEG,414
2,2,2.JPEG,493
3,3,3.JPEG,17
4,4,4.JPEG,388
...,...,...,...
10009,10009,10009.JPEG,235
10010,10010,10010.JPEG,191
10011,10011,10011.JPEG,466
10012,10012,10012.JPEG,258


In [ ]:
test_data1.to_csv("/data/ephemeral/home/jinbong/output_clip.csv", index=False)
test_data1

,ID,image_path,target
0,0,0.JPEG,328
1,1,1.JPEG,414
2,2,2.JPEG,493
3,3,3.JPEG,17
4,4,4.JPEG,388
...,...,...,...
10009,10009,10009.JPEG,235
10010,10010,10010.JPEG,191
10011,10011,10011.JPEG,466
10012,10012,10012.JPEG,258
